In [4]:
import pandas as pd
import ydata_profiling as pp
from pycaret.regression import *
pd.set_option('display.max_columns', None)

In [61]:
dataset = pd.read_csv("data_properti_filtered.csv")
dataset

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,3500000000,201,113,3.0,4,1,Jakarta,Jakarta Timur,Matraman,3
1,38000000000,600,1028,5.0,4,4,Jakarta,Jakarta Pusat,Kemayoran,3
2,775000000,110,45,3.0,3,1,Jakarta,Jakarta Barat,Cengkareng,3
3,1300000000,140,70,3.0,2,0,Jakarta,Jakarta Pusat,Johar Baru,3
4,49000000000,429,490,3.0,3,4,Jakarta,Jakarta Pusat,Menteng,3
...,...,...,...,...,...,...,...,...,...,...
21865,90000000000,800,1125,4.0,4,0,Jakarta,Jakarta Pusat,Menteng,3
21866,46000000000,544,886,4.0,4,0,Jakarta,Jakarta Pusat,Menteng,3
21867,7000000000,450,312,4.0,3,2,Jakarta,Jakarta Barat,Kebon Jeruk,3
21868,30450000000,450,783,6.0,4,2,Jakarta,Jakarta Pusat,Menteng,3


In [6]:
dataset.head(100)

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,850000000,100,50,3.0,2,0,Jawa Timur,Surabaya,Tambaksari,0
1,1800000000,150,180,3.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,0
2,6000000000,350,385,6.0,4,0,Jawa Timur,Sidoarjo,Sidoarjo,0
3,550000000,70,80,2.0,2,0,Jawa Timur,Surabaya,Asemrowo,0
4,1499000000,122,88,3.0,3,0,Jawa Timur,Surabaya,Mulyorejo,0
...,...,...,...,...,...,...,...,...,...,...
95,5500000000,300,458,6.0,4,0,Jawa Timur,Surabaya,Gayungan,0
96,3300000000,504,391,5.0,5,0,Jawa Timur,Surabaya,Kenjeran,0
97,235000000,36,60,2.0,1,0,Jawa Timur,Malang,Wagir,0
98,1900000000,155,82,4.0,4,0,Jawa Timur,Gresik,Manyar,0


In [7]:
prov = dataset[dataset['province'] == 'Sulawesi Selatan']
prov['city'].value_counts()

Series([], Name: city, dtype: int64)

In [62]:
data_train = dataset.sample(frac=0.8, random_state=110)
data_unseen = dataset.drop(data_train.index)

data_train.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for train: ' + str(data_train.shape))
print('Data for test: ' +str(data_unseen.shape))

Data for train: (17496, 10)
Data for test: (4374, 10)


In [63]:
cat_features = ['province', 'city']
exp = setup(data_train, target = 'price', categorical_features=cat_features, 
            ignore_features = ['z', 'district', 'latitude', 'longitude'], 
            numeric_imputation = 'mean', categorical_imputation= 'mode', 
            remove_outliers= True, normalize = True)

,Description,Value
0,Session id,5916
1,Target,price
2,Target type,Regression
3,Original data shape,"(17496, 10)"
4,Transformed data shape,"(16883, 13)"
5,Transformed train set shape,"(11634, 13)"
6,Transformed test set shape,"(5249, 13)"
7,Ignore features,4
8,Numeric features,6
9,Categorical features,2


In [64]:
training_data = get_config(variable='X_train')
training_data

,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,cluster
14390,195,96,3.0,3,0,Jakarta,Jakarta Selatan,3
1412,60,56,3.0,2,0,Jakarta,Jakarta Selatan,3
6527,125,100,3.0,2,0,Jakarta,Jakarta Timur,3
7102,500,626,5.0,3,4,Jakarta,Jakarta Selatan,3
13077,102,102,4.0,3,0,Jakarta,Jakarta Utara,3
...,...,...,...,...,...,...,...,...
17368,200,195,5.0,3,1,Jakarta,Jakarta Timur,3
13208,700,210,5.0,5,1,Jakarta,Jakarta Selatan,3
10980,200,153,4.0,3,0,Jakarta,Jakarta Utara,3
11725,120,209,3.0,2,0,Jakarta,Jakarta Utara,3


In [65]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,2758629788.8002,50444325209587597312.0000,7074481874.8074,0.6450,0.4580,0.8305,1.6570
et,Extra Trees Regressor,2720481806.9672,51182756397959348224.0000,7125915957.3853,0.6411,0.4587,0.7491,1.4410
lightgbm,Light Gradient Boosting Machine,2970630341.9604,52928264338477809664.0000,7261136560.4884,0.6262,0.4867,0.8639,0.7180
gbr,Gradient Boosting Regressor,3099374406.3780,55304464733075726336.0000,7404522930.3296,0.6101,0.5199,0.9783,0.9360
catboost,CatBoost Regressor,2997658765.1604,55899684489131163648.0000,7451807511.8284,0.6068,0.4899,0.8305,3.7610
xgboost,Extreme Gradient Boosting,3053324974.5307,60561450331820130304.0000,7756931826.1278,0.5730,0.4969,0.8778,0.5840
knn,K Neighbors Regressor,3316225349.9849,66952636594114134016.0000,8150397792.4346,0.5296,0.5236,0.8630,0.3990
ada,AdaBoost Regressor,4582543141.0203,77675344610478243840.0000,8800375536.8935,0.4504,0.7315,1.5702,0.4640
dt,Decision Tree Regressor,3332802761.0743,88843863886178877440.0000,9381747765.3689,0.3723,0.5762,0.8639,0.3580
dummy,Dummy Regressor,6191358294.4163,143716939377184358400.0000,11971318181.9856,-0.0107,1.1716,2.3092,0.3780


In [66]:
#evaluate precition recall curve 
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [67]:
predict_model(best_model, data = data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,2647219839.9961,47081341897181143040.0000,6861584503.3914,0.6465,0.4611,1.6013


,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster,price,prediction_label
0,110,60,3.0,3,2,Jakarta,Jakarta Selatan,Tebet,3,1285000000,1.288059e+09
1,100,108,3.0,2,0,Jakarta,Jakarta Timur,Kramat Jati,3,1600000000,2.261430e+09
2,150,120,3.0,2,0,Jakarta,Jakarta Timur,Cakung,3,1750000000,1.681117e+09
3,116,112,6.0,4,2,Jakarta,Jakarta Timur,Cakung,3,3243900000,2.418479e+09
4,150,105,3.0,3,0,Jakarta,Jakarta Selatan,Jagakarsa,3,1675000000,1.939764e+09
...,...,...,...,...,...,...,...,...,...,...,...
4369,62,90,1.0,1,0,Jakarta,Jakarta Timur,Cakung,3,1125000000,1.486150e+09
4370,300,250,4.0,4,4,Jakarta,Jakarta Barat,Kembangan,3,3600000000,5.889252e+09
4371,1000,600,4.0,4,0,Jakarta,Jakarta Barat,Kembangan,3,18500000000,3.684650e+10
4372,400,285,4.0,4,1,Jakarta,Jakarta Pusat,Menteng,3,28000000000,1.853432e+10


In [68]:
finalize_model(best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['buildingSize', 'landSize',
                                             'bedRooms', 'bathRooms', 'garages',
                                             'cluster'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['province', 'city'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWr...nclude=['province', 'city'],
                                    transformer=OneHotEncoder(cols=['province',
                                                                    'city'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('remove_outliers',
                 TransformerWrapper(transformer=RemoveOutliers(random_state=5916))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator',
                 RandomForestRegressor(n_jobs=-1, random_state=5916))])

In [69]:
save_model(best_model, 'property_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['buildingSize', 'landSize',
                                              'bedRooms', 'bathRooms', 'garages',
                                              'cluster'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['province', 'city'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWr...r(include=['province', 'city'],
                                     transformer=OneHotEncoder(cols=['province',
                                                                     'city'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),